In [1]:
print("feature store")

feature store


In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("../../data/Housing_processed.csv")

print(df.head())

      price      area  bedrooms  bathrooms   stories  mainroad  guestroom  \
0  13300000  1.046726  1.403419   1.421812  1.378217         1          0   
1  12250000  1.757010  1.403419   5.405809  2.532024         1          0   
2  12250000  2.218232  0.047278   1.421812  0.224410         1          0   
3  12215000  1.083624  1.403419   1.421812  0.224410         1          0   
4  11410000  1.046726  1.403419  -0.570187  0.224410         1          1   

   basement  hotwaterheating  airconditioning   parking  prefarea  \
0         0                0                1  1.517692         1   
1         0                0                1  2.679409         0   
2         1                0                0  1.517692         1   
3         1                0                1  2.679409         1   
4         1                0                1  1.517692         0   

   furnishingstatus  
0                 0  
1                 0  
2                 1  
3                 0  
4           

In [6]:
timestamps = pd.date_range(
    end=pd.Timestamp.now(),
    start=pd.Timestamp.now(),
    periods=len(df),
    freq=None
).to_frame(name="event_timestamp", index=False)

df['event_timestamp'] = timestamps['event_timestamp']
df['house_id'] = range(1, len(df) + 1)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus,event_timestamp,house_id
0,13300000,1.046726,1.403419,1.421812,1.378217,1,0,0,0,1,1.517692,1,0,2025-12-24 18:08:04.022308000,1
1,12250000,1.757010,1.403419,5.405809,2.532024,1,0,0,0,1,2.679409,0,0,2025-12-24 18:08:04.022307904,2
2,12250000,2.218232,0.047278,1.421812,0.224410,1,0,1,0,0,1.517692,1,1,2025-12-24 18:08:04.022307808,3
3,12215000,1.083624,1.403419,1.421812,0.224410,1,0,1,0,1,2.679409,1,0,2025-12-24 18:08:04.022307713,4
4,11410000,1.046726,1.403419,-0.570187,0.224410,1,1,1,0,1,1.517692,0,0,2025-12-24 18:08:04.022307617,5


In [9]:
X = df.drop(columns=["price"])
y = pd.DataFrame(df[["price", "event_timestamp", "house_id"]])

print("\nFeatures")
print(X.head())
print("\nTarget")
print(y.head())

Features
       area  bedrooms  bathrooms   stories  mainroad  guestroom  basement  \
0  1.046726  1.403419   1.421812  1.378217         1          0         0   
1  1.757010  1.403419   5.405809  2.532024         1          0         0   
2  2.218232  0.047278   1.421812  0.224410         1          0         1   
3  1.083624  1.403419   1.421812  0.224410         1          0         1   
4  1.046726  1.403419  -0.570187  0.224410         1          1         1   

   hotwaterheating  airconditioning   parking  prefarea  furnishingstatus  \
0                0                1  1.517692         1                 0   
1                0                1  2.679409         0                 0   
2                0                0  1.517692         1                 1   
3                0                1  2.679409         1                 0   
4                0                1  1.517692         0                 0   

                event_timestamp  house_id  
0 2025-12-24 18:08:04

In [10]:
import sqlalchemy as db
from sqlalchemy import text
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

connstr = os.getenv("POSTGRE_SQL_CONN_STR")
offline_db = os.getenv("POSTGRE_SQL_OFFLINE_DB")
full_connstr = f"{connstr}/{offline_db}"

engine = db.create_engine(full_connstr)


with engine.connect() as conn:
    result = conn.execute(text("SELECT current_database(), current_user"))
    print(result.fetchone())

('feast_offline', 'postgres')


In [11]:
X.to_sql('house_features_sql', engine, if_exists='replace', index=False)
y.to_sql('house_target_sql', engine, if_exists='replace', index=False)

-1

In [13]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../../", "data"))
X.to_parquet(path=os.path.join(parent_dir, 'house_features.parquet'), index=False)
y.to_parquet(path=os.path.join(parent_dir, 'house_target.parquet'), index=False)

In [14]:
print(X.head())

       area  bedrooms  bathrooms   stories  mainroad  guestroom  basement  \
0  1.046726  1.403419   1.421812  1.378217         1          0         0   
1  1.757010  1.403419   5.405809  2.532024         1          0         0   
2  2.218232  0.047278   1.421812  0.224410         1          0         1   
3  1.083624  1.403419   1.421812  0.224410         1          0         1   
4  1.046726  1.403419  -0.570187  0.224410         1          1         1   

   hotwaterheating  airconditioning   parking  prefarea  furnishingstatus  \
0                0                1  1.517692         1                 0   
1                0                1  2.679409         0                 0   
2                0                0  1.517692         1                 1   
3                0                1  2.679409         1                 0   
4                0                1  1.517692         0                 0   

                event_timestamp  house_id  
0 2025-12-24 18:08:04.02230800

In [15]:
print(X.tail())

         area  bedrooms  bathrooms   stories  mainroad  guestroom  basement  \
540 -0.991879 -1.308863  -0.570187 -0.929397         1          0         1   
541 -1.268613  0.047278  -0.570187 -0.929397         0          0         0   
542 -0.705921 -1.308863  -0.570187 -0.929397         1          0         0   
543 -1.033389  0.047278  -0.570187 -0.929397         0          0         0   
544 -0.599839  0.047278  -0.570187  0.224410         1          0         0   

     hotwaterheating  airconditioning   parking  prefarea  furnishingstatus  \
540                0                0  1.517692         0                 2   
541                0                0 -0.805741         0                 1   
542                0                0 -0.805741         0                 2   
543                0                0 -0.805741         0                 0   
544                0                0 -0.805741         0                 2   

                  event_timestamp  house_id  
540 